In [1]:
import cv2
import torch
from torchvision.transforms import transforms
from torchvision.models import resnet50

In [2]:
model = torch.hub.load("intel-isl/MiDaS", "MiDaS_small")

Using cache found in C:\Users\ludwig/.cache\torch\hub\intel-isl_MiDaS_master
e:\repos\rebelway-applied-ml\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\repos\rebelway-applied-ml\.venv\Lib\site-packages\timm\models\layers\__init__.py:49: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Loading weights:  None


Using cache found in C:\Users\ludwig/.cache\torch\hub\rwightman_gen-efficientnet-pytorch_master


In [ ]:
model.eval()
device = "cuda"
model.to(device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((384,384)),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[.0229,0.224,0.225])
])
source = "./person_face_640.mp4"
cap = cv2.VideoCapture(source)

if not cap.isOpened():
    print("error reading source video")
    exit()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("error reading frame")
        break
    input_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_image = transform(input_image).unsqueeze(0).to(device)
    # depth estimation
    with torch.no_grad():
        depth_map = model(input_image)

    frame_height, frame_width = frame.shape[:2]
    depth_map = depth_map.squeeze().cpu().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    depth_map = cv2.resize(depth_map, (frame_width, frame_height))

    print("Depth map shape:", depth_map.shape)
    cv2.imshow("Depth map", depth_map)
    cv2.imshow("Feed", frame)

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
    

error reading frame
